In [6]:
from confluent_kafka import Producer

conf = {
    'bootstrap.servers': 'b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092',  # This is the address of your Kafka broker(s)
    'client.id': 'retail_store_producer',
    'error_cb': lambda err: print(f"Error: {err}"),  # Error callback function
    'retries': 5,  # Number of retries
    'retry.backoff.ms': 1000  # Wait time between retries in milliseconds
}
producer = Producer(conf)

In [7]:
transaction_data = {
    'product_name': 'rocket_ship',
    'category': 'toy',
    'amount': 10
}

# Convert the data to string format; often JSON is used
import json
transaction_string = json.dumps(transaction_data)

# Write to the 'retail_products' topic (ledger)
producer.produce('retail_products', transaction_string)

# Poll the producer for events, including delivery reports
# This is important for the delivery callback to be triggered
producer.poll(0)

# Ensure the data is sent
producer.flush()

0

0 means the message wasn't written.

In [8]:
def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

# Use the callback when producing a message
producer.produce('retail_products', transaction_string, callback=delivery_report)
producer.flush()

Message delivery failed: KafkaError{code=NOT_ENOUGH_REPLICAS,val=19,str="Broker: Not enough in-sync replicas"}


0

In [5]:
from confluent_kafka import Consumer, KafkaError

PERSONAL_ID = input("Enter your name or id:" ) # To ensure all users read the entire topic

conf = {
    'bootstrap.servers': 'b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092',  # Address of your Kafka broker(s)
    'group.id': f'retail_store_group_{PERSONAL_ID}',
    'auto.offset.reset': 'earliest'  # Start reading from the beginning of the topic
}

consumer = Consumer(conf)
consumer.subscribe(['retail_products'])

In [ ]:
while True:
    msg = consumer.poll(1.0)
    if msg is None:
        continue
    if msg.error():
        if msg.error().code() == KafkaError._PARTITION_EOF:
            continue
        else:
            print(msg.error())
            break

    print('Received message: {}'.format(msg.value().decode('utf-8')))

def shutdown():
    consumer.close()

import atexit
atexit.register(shutdown)

%4|1702653105.139|MAXPOLL|rdkafka#consumer-1| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 494ms (adjust max.poll.interval.ms for long-running message processing): leaving group
